## Making a locations lookup table for subway stops from the turnstile data.

The station locations file was found at 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_200919.txt'

The data is older (2009) and does not include the PTH, RIT and some other divisions that made up a small part of the turnstile dataset.

Also, the station names and lines are only partial matches between the two datasets.

In [1]:
import pandas as pd
import numpy as np
import pickle

### Import any week of the turnstile data, just to get the station information

In [2]:
df=pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_200919.txt')

### Get the station information "STATION", "LINENAME" and "DIVISION" from the turnstile df

In [3]:
df4 = df.groupby(['STATION', 'LINENAME', 'DIVISION'])[['SCP']].count()
df6 = pd.DataFrame(list(df4.index))
df6.columns=["STATION", "LINENAME", "DIVISION"]
df6

,STATION,LINENAME,DIVISION
0,1 AV,L,BMT
1,103 ST,1,IRT
2,103 ST,6,IRT
3,103 ST,BC,IND
4,103 ST-CORONA,7,IRT
...,...,...,...
480,WOODLAWN,4,IRT
481,WORLD TRADE CTR,ACE23,IND
482,WTC-CORTLANDT,1,IRT
483,YORK ST,F,IND


### Import the station location dataset

In [4]:
locations = pd.read_csv('http://web.mta.info/developers/data/nyct/subway/Stations.csv')

### We realized that there are actually many stations with the same name, but different locations data. Including the lines, or 'Daytime Routes' column in the locations dataset was the best way to identify individual stations.

In [5]:
locations['Stop Name'].value_counts()

Canal St                   6
86 St                      6
23 St                      5
Fulton St                  5
125 St                     4
                          ..
World Trade Center         1
Harlem - 148 St            1
E 143 St - St Mary's St    1
168 St - Washington Hts    1
42 St - Bryant Pk          1
Name: Stop Name, Length: 377, dtype: int64

In [6]:
locations.groupby(['Stop Name','Division'])['GTFS Latitude'].count().value_counts()

1    402
2     41
3      4
Name: GTFS Latitude, dtype: int64

In [7]:
locations.groupby(['Stop Name','Daytime Routes'])['GTFS Latitude'].count().value_counts()

1    496
Name: GTFS Latitude, dtype: int64

Stations in location dataset are lower case, but in the turnstile data are uppercase, so the station name in locations dataset is changed to uppercase

In [8]:
locations["STATION"] = locations['Stop Name'].str.upper()

The lines (Daytime Routes) in the locations dataset have spaces but in the turnstile data lines (LINENAMES) don't, so the spaces in locations dataset lines are removed

In [9]:
locations["Daytime Routes"] = locations["Daytime Routes"].str.replace(" ","")

### Using character set percent overlap to match locations dataframe stations to turnstile dataframe stations

Using pandas merge function did not work well because of the partial matches between the 'STATIONS' column in the turnstile dataset and the 'STATIONS' column in the locations dataset, as well as the partial matches between the 'LINENAME' column in the turnstile dataset and the 'Daytime Routes' column in the locations dataset.

A way to do partial matching is to find the maximum percent overlap of the characters in a string with the other strings. First, lists of sets of the characters in each line name and each linename were made for the datasets. 

In [10]:
#making a list of sets for each station, containing the characters in its name
df6_sets=[]
for i in df6["STATION"]:
    station_chars = set(i)
    df6_sets.append(station_chars)
    
locations_sets=[]
for i in locations["STATION"]:
    station_chars = set(i)
    locations_sets.append(station_chars)

In [11]:
#making a list of sets for each linename, containing the characters in its name
df6_line_sets=[]
for i in df6["LINENAME"]:
    line_chars = set(i)
    df6_line_sets.append(line_chars)
    
locations_line_sets=[]
for i in locations["Daytime Routes"]:
    line_chars = set(i)
    locations_line_sets.append(line_chars)

A loop was made to look at the characters from each station name in the turnstile dataset and find the best matches (% overlap) in the locations dataset. 
Then, if there are multiple equally good station character matches, it looks at the overlap between the linename characters in the turnstile datset and the linename characters in the locations dataset. It chooses the best match here.


In retrospect, this was probably a waste of time since there were only ~500 stations and I could have organized the data by hand more quickly.

In [12]:
def best_station_match():
    """
    A function that finds and returns the index of the locations dataset that most likely matches 
    each station in the turnstile dataset based on character overlap for stations and lines
    """
    best_match_list =[]
    pos=-1
    #loop over the list of sets of characters in stations from the turnstile data
    for i in df6_sets:
        pos += 1
        best_matches=[]
        max_score=0
        
        #find the best match by % overlap of this set with the station character sets from locations 
        for j in locations_sets:
            match_score = len(i & j)/max(len(i),len(j))
            if match_score > max_score:
                max_score = match_score
                best_match = locations_sets.index(j)             
        best_score= len(locations_sets[best_match] & i)/max(len(locations_sets[best_match]),len(i))
        pos2=-1
        
        #find other station character matches if they were equally as good as the best match
        for j in locations_sets:
            pos2+=1
            if len(j & i)/max(len(i),len(j)) == best_score:
                best_matches.append(pos2)
                
        #If there is only one best station name match, assign this one
        if len(best_matches)==1:
            best_matches=best_matches[0]
            best_match_list.append(best_matches)
            continue
        
        #If there is more than one best station match, look at each line and assign the best line match
        if len(best_matches) > 1:
            best_match_2=0
            max_score_2=0
            k = df6_line_sets[pos]
            for z in best_matches:
                l = locations_line_sets[z]
                match_score_2 = len(k & l)/max(len(k),len(l))
                if match_score_2 > max_score_2:
                    max_score_2 = match_score_2
                    best_match_2=z
            best_matches=best_match_2                  
            best_match_list.append(best_matches)
                
            
    return best_match_list



In [13]:
test = best_station_match()

In [14]:
index_needed = list(test)

Append the best match lines from locations dataframe to the turnstile stations names dataframe

In [15]:
final = pd.concat([df6,locations.iloc[index_needed].reset_index()],axis=1)

In [16]:
#pd.set_option('display.max_rows', 500)
final

,STATION,LINENAME,DIVISION,index,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,STATION
0,1 AV,L,BMT,118,119,119,L06,BMT,Canarsie,1 Av,M,L,Subway,40.730953,-73.981628,8 Av,Brooklyn,1 AV
1,103 ST,1,IRT,309,309,309,119,IRT,Broadway - 7Av,103 St,M,1,Subway,40.799446,-73.968379,Uptown & The Bronx,Downtown,103 ST
2,103 ST,6,IRT,395,395,395,624,IRT,Lexington Av,103 St,M,6,Subway,40.790600,-73.947478,Uptown & The Bronx,Downtown,103 ST
3,103 ST,BC,IND,155,156,156,A18,IND,8th Av - Fulton St,103 St,M,BC,Subway,40.796092,-73.961454,Uptown & The Bronx,Downtown & Brooklyn,103 ST
4,103 ST-CORONA,7,IRT,450,450,450,706,IRT,Flushing,103 St - Corona Plaza,Q,7,Elevated,40.749865,-73.862700,Flushing,Manhattan,103 ST - CORONA PLAZA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,WOODLAWN,4,IRT,378,378,378,401,IRT,Jerome Av,Woodlawn,Bx,4,Elevated,40.886037,-73.878751,NaN,Manhattan,WOODLAWN
481,WORLD TRADE CTR,ACE23,IND,171,171,624,E01,IND,8th Av - Fulton St,World Trade Center,M,E,Subway,40.712582,-74.009781,Uptown & Queens,NaN,WORLD TRADE CENTER
482,WTC-CORTLANDT,1,IRT,328,328,328,138,IRT,Broadway - 7Av,WTC Cortlandt,M,1,Subway,40.711835,-74.012188,Uptown & The Bronx,Downtown,WTC CORTLANDT
483,YORK ST,F,IND,235,235,235,F18,IND,6th Av - Culver,York St,Bk,F,Subway,40.701397,-73.986751,Manhattan,Coney Island,YORK ST


In [17]:
final.isnull().sum()

STATION                   0
LINENAME                  0
DIVISION                  0
index                     0
Station ID                0
Complex ID                0
GTFS Stop ID              0
Division                  0
Line                      0
Stop Name                 0
Borough                   0
Daytime Routes            0
Structure                 0
GTFS Latitude             0
GTFS Longitude            0
North Direction Label    33
South Direction Label     7
STATION                   0
dtype: int64

### Make some corrections manually

Looking at the dataset, there were some obvious errors (such as cases with very different station names, and incorrect matches for PTH line that was not part of the locations data) that had to be corrected by hand. There are some remaining errors.

In [ ]:
#final.to_csv("stations_locations.csv")

In [31]:
#locations.to_csv("locations.csv")

In [19]:
#final = pd.read_csv('stations_final_2.csv')

### Pickle this final data frame

In [ ]:
#with open("station_locations_correct_2.pickle","wb") as write_file:
#    pickle.dump(final, write_file)

This pickle object is used in the main notebook to assign gps locations and borough information to stations for some graphs. 

The gps data could also have been used to find the busiest stations within some distance range from tech universities and tech hubs, using gps locations for those places and calculating euclidean distance